While the Data Submission Portal API is primarily intended to support submissions, the information that can be returned is also useful for reporting an monitoring a collection of submissions.  In the notebooks, we'll demonstrate some of the queries that can provide information useful for seeing the current state of your submisison universe

In [1]:
import requests
import pandas as pd
import os
from IPython.display import display, Markdown, Latex
from datetime import datetime, timezone
import plotly.express as px
import numpy as np

In [2]:
prod = 'https://hub.datacommons.cancer.gov/api/graphql'
#Note that use of Dev2 is for example purposes only.  Unless you are an approved tester, you should be using the production URL
dev2 = 'https://hub-dev2.datacommons.cancer.gov/api/graphql'
stage = 'https://hub-stage.datacommons.cancer.gov/api/graphql'

In [ ]:
def apiQuery(tier, query, variables):
    if tier == 'dev2':
        token = os.environ['DEV2API']
        url = dev2
    elif tier == 'stage':
        token = os.environ['STAGEAPI']
        url = stage
    
    headers = {"Authorization": f"Bearer {token}"}
    try:
        if variables is None:
            result = requests.post(url = url, headers = headers, json={"query": query})
        else:
            result = requests.post(url = url, headers = headers, json = {"query":query, "variables":variables})
        if result.status_code == 200:
            return result.json()
        else:
            print(f"Error: {result.status_code}")
            return result.content
    except requests.exceptions.HTTPError as e:
        return(f"HTTP Error: {e}")

One potentially useful report would be a simple overview of all the studies your organization is working on.  This can be done with the *listApprovedStudiesOfMyOrganization* query.

In [4]:
study_query = """
{
  listApprovedStudiesOfMyOrganization{
    originalOrg
    studyName
    studyAbbreviation
    dbGaPID
    controlledAccess
  }
}
"""

In [5]:
studies_res = apiQuery('stage',study_query,None)
studies_df = pd.DataFrame(studies_res['data']['listApprovedStudiesOfMyOrganization'])
display(Markdown(studies_df.to_markdown()))

|    | originalOrg   | studyName               | studyAbbreviation       |   dbGaPID | controlledAccess   |
|---:|:--------------|:------------------------|:------------------------|----------:|:-------------------|
|  0 | FNL           | G_OpenStudy_Stage       | G_OpenStudy_Stage       |           | False              |
|  1 | FNL           | G_controlledStudy_Stage | G_controlledStudy_Stage |     34424 | True               |

While this result provides a high-level overview, it doesn't really dig into the details of what's really going on.  To get to a more fine-grained look at the actual submissions, the *listSubmissions* query can delve into much detail. Note that the *listSubmissions* query requires the **status** parameter which can be one of the following:
- All
- New
- In Progress
- Submitted
- Released
- Completed
- Archived
- Canceled
- Rejected
- Withdrawn
- Deleted

Since we're reporting on the overall state of all submissions, we'll use **All** in the query.

In [6]:
listsubmissions_query = """
query ListSubmissions($status: String!){
  listSubmissions(status: $status){
    submissions{
      _id
      name
      submitterID
      submitterName
      studyAbbreviation
      studyID
      dbGaPID
      createdAt
      updatedAt
      metadataValidationStatus
      fileValidationStatus
      status
    }
  }
}
"""

In [7]:
submisisons_variable = {"status":"All"}

In [8]:
submission_res = apiQuery('stage',listsubmissions_query,submisisons_variable)
submission_df = pd.DataFrame(submission_res['data']['listSubmissions']['submissions'])
display(Markdown(submission_df.to_markdown()))

|    | _id                                  | name                      | submitterID                          | submitterName   | studyAbbreviation       | studyID                              | dbGaPID   | createdAt                | updatedAt                | metadataValidationStatus   | fileValidationStatus   | status      |
|---:|:-------------------------------------|:--------------------------|:-------------------------------------|:----------------|:------------------------|:-------------------------------------|:----------|:-------------------------|:-------------------------|:---------------------------|:-----------------------|:------------|
|  0 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Stage API Submission Test | d2a62f73-03c7-41cc-ad7a-245c1364625b | Todd Pihl       | G_controlledStudy_Stage | 4a78e24f-7cd5-4f3c-a3dc-87ff02e42ecd | 34424     | 2024-10-18T20:02:36.282Z | 2024-10-21T14:26:40.081Z | Error                      |                        | In Progress |
|  1 | 28f08b68-4b80-40b4-bc25-01c17e4b9ef0 | IRCOCO Demo               | d2a62f73-03c7-41cc-ad7a-245c1364625b | Todd Pihl       | G_OpenStudy_Stage       | 5bdf7aa6-9d52-4144-9cdc-ad4e1987c696 | phs000123 | 2024-10-11T17:13:53.703Z | 2024-10-11T17:21:09.262Z | Error                      |                        | In Progress |

## Submission status
It's useful to get a quick overview of how many submissions are in the various states.

In [9]:
print(submission_df['status'].value_counts())

status
In Progress    2
Name: count, dtype: int64


## Submission Inactivity
One of the states that submitters need to pay attention to is the inactivity time on submissions that are in the *In Progress* or *New* states.  Any submission in those states that is inactive for 60 days will start receiving warnings and any submissions in those states that is inactive for 180 days will be deleted.  So having a dashboard of inactivity may help with spotting submissions that are in trouble before they get deleted.

Lets create three bins of submissions, those that have been inactive for <60 days, those that have been inactive for >60 days but <150, and then those that are at risk and are >150.

In [10]:
submission_df['createdAt'] = pd.to_datetime(submission_df['createdAt'])
submission_df['updatedAt'] = pd.to_datetime(submission_df['updatedAt'])

In [11]:
days = []
for index, row in submission_df.iterrows():
    update = row['updatedAt']
    now = datetime.now(timezone.utc)
    diff = (now - update).days
    days.append(diff)
submission_df.insert(8,'inactiveDays',days,True)

In [12]:
display(Markdown(submission_df.to_markdown()))

|    | _id                                  | name                      | submitterID                          | submitterName   | studyAbbreviation       | studyID                              | dbGaPID   | createdAt                        |   inactiveDays | updatedAt                        | metadataValidationStatus   | fileValidationStatus   | status      |
|---:|:-------------------------------------|:--------------------------|:-------------------------------------|:----------------|:------------------------|:-------------------------------------|:----------|:---------------------------------|---------------:|:---------------------------------|:---------------------------|:-----------------------|:------------|
|  0 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Stage API Submission Test | d2a62f73-03c7-41cc-ad7a-245c1364625b | Todd Pihl       | G_controlledStudy_Stage | 4a78e24f-7cd5-4f3c-a3dc-87ff02e42ecd | 34424     | 2024-10-18 20:02:36.282000+00:00 |             14 | 2024-10-21 14:26:40.081000+00:00 | Error                      |                        | In Progress |
|  1 | 28f08b68-4b80-40b4-bc25-01c17e4b9ef0 | IRCOCO Demo               | d2a62f73-03c7-41cc-ad7a-245c1364625b | Todd Pihl       | G_OpenStudy_Stage       | 5bdf7aa6-9d52-4144-9cdc-ad4e1987c696 | phs000123 | 2024-10-11 17:13:53.703000+00:00 |             24 | 2024-10-11 17:21:09.262000+00:00 | Error                      |                        | In Progress |

In [13]:
submfig = px.histogram(submission_df, x='inactiveDays', nbins=3)
submfig.show()

Alternatively, it might be worth looking across the entire timeframe.

In [14]:
counts, bins = np.histogram(submission_df.inactiveDays, bins=range(0,180,20))
bins = 0.5 * (bins[:-1] + bins[1:])
subfig = px.bar(x=bins, y=counts, labels={'x':"Inactive Days", 'y':"Count"}, color=bins)
subfig.show()

Beyond looking at the overall status of submissions, the API can also be useful for generating reports on the error states of specific submissions.  When there are validation errors, there are frequently large numbers of them, and a quick summary report can highlight areas that need the most attention.  Generating this kind of report is straight-forward.  

First, find out which submissions have errors.  This can be done by adjusting the *listSubmissions* query we ran above.  Specifically, rather than requesting all submissions, we'll get just the ones that are **In Progress** since those would be the submissions that might have errors.  We'll also select for just the submissions with a **metadataValidationStatus** of **Error**.

In [15]:
submission_variables = {"status":"In Progress"}

In [16]:
inprog_res = apiQuery('stage',listsubmissions_query,submisisons_variable)
inprog_df = pd.DataFrame(inprog_res['data']['listSubmissions']['submissions'])
error_df = inprog_df[inprog_df['metadataValidationStatus'] == 'Error']
display(Markdown(error_df.to_markdown()))

|    | _id                                  | name                      | submitterID                          | submitterName   | studyAbbreviation       | studyID                              | dbGaPID   | createdAt                | updatedAt                | metadataValidationStatus   | fileValidationStatus   | status      |
|---:|:-------------------------------------|:--------------------------|:-------------------------------------|:----------------|:------------------------|:-------------------------------------|:----------|:-------------------------|:-------------------------|:---------------------------|:-----------------------|:------------|
|  0 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Stage API Submission Test | d2a62f73-03c7-41cc-ad7a-245c1364625b | Todd Pihl       | G_controlledStudy_Stage | 4a78e24f-7cd5-4f3c-a3dc-87ff02e42ecd | 34424     | 2024-10-18T20:02:36.282Z | 2024-10-21T14:26:40.081Z | Error                      |                        | In Progress |
|  1 | 28f08b68-4b80-40b4-bc25-01c17e4b9ef0 | IRCOCO Demo               | d2a62f73-03c7-41cc-ad7a-245c1364625b | Todd Pihl       | G_OpenStudy_Stage       | 5bdf7aa6-9d52-4144-9cdc-ad4e1987c696 | phs000123 | 2024-10-11T17:13:53.703Z | 2024-10-11T17:21:09.262Z | Error                      |                        | In Progress |

For the purposes of this demo, we'll just focus on one of the submissions, the **Jupyter Demo 4** submission.  We'll also re-use the QC query from the *DataHubAPIDemo* workbook since that brings back what we need.  Finallly, we'll just look at **Errors** since those prevent submission.  It's highly recommended that you also try to correct **Warnings** but those do not block submissions.

In [17]:
qc_check_query = """
query GetQCResults(
  $id: ID!
  $severities: String
  $first: Int
){
  submissionQCResults(_id:$id, severities:$severities, first:$first){
    total
    results{
      submissionID
      severity
      type
      errors{
        title
        description
      }
    }
  }
}
"""

In [19]:
subname = "Stage API Submission Test"
submission_id = error_df.loc[error_df['name'] == subname, '_id'].iloc[0]

In [20]:
qc_variables = {"id":submission_id, "severities":"Error", "first":-1}

In [23]:
suberrors_res = apiQuery('stage',qc_check_query,qc_variables)
suberrors_df = pd.DataFrame(suberrors_res['data']['submissionQCResults']['results'])
display(Markdown(suberrors_df.to_markdown()))

|     | submissionID                         | severity   | type         | errors                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
|----:|:-------------------------------------|:-----------|:-------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|   0 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | study        | [{'title': 'Missing required property', 'description': '[Demo_study.tsv: line 2] Required property "file_types_and_format" is empty.'}, {'title': 'Missing required property', 'description': '[Demo_study.tsv: line 2] Required property "study_access" is empty.'}, {'title': 'Missing required property', 'description': '[Demo_study.tsv: line 2] Required property "study_version" is empty.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|   1 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 93] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|   2 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 133] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|   3 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 142] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|   4 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 139] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|   5 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 141] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|   6 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 138] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|   7 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 92] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|   8 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 56] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|   9 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 9] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  10 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 6] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  11 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 95] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  12 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 94] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  13 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 91] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  14 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 55] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  15 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 90] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  16 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 54] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  17 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 8] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  18 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 5] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  19 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 89] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  20 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 53] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  21 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 88] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  22 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 52] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  23 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 18] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  24 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 15] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  25 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 19] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  26 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 16] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  27 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 51] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  28 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 50] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  29 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 87] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  30 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 131] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  31 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 12] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  32 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 10] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  33 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 11] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  34 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 13] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  35 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 148] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  36 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 136] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  37 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 135] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  38 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 140] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  39 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 137] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  40 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 157] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  41 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 156] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  42 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 132] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  43 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 130] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  44 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 134] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  45 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 85] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  46 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 48] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  47 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 86] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  48 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 49] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  49 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 84] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  50 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 47] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  51 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 118] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  52 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 105] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  53 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 119] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  54 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 106] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  55 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 120] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  56 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 121] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  57 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 83] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  58 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 46] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  59 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 82] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  60 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 45] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  61 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 81] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  62 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 44] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  63 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 80] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  64 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 43] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  65 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 79] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  66 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 78] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  67 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 42] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  68 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 41] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  69 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 77] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  70 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 40] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  71 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 123] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  72 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 122] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  73 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 39] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  74 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 20] "DNA" is not a permissible value for property “sample_type”.'}, {'title': 'Many-to-one relationship conflict', 'description': '"[Demo_sampleFIXED.tsv: line 20]": associated with multiple “participant” nodes: ["79NTL6", "G04XZJ"].'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|  75 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 75] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  76 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 38] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  77 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 74] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  78 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 73] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  79 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 37] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  80 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 72] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  81 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 36] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  82 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 17] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  83 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 14] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  84 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 129] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  85 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 127] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  86 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 128] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  87 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 125] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  88 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 126] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  89 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 124] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  90 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 117] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  91 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 104] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  92 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 116] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  93 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 103] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  94 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 102] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  95 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 115] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  96 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 113] "DNA" is not a permissible value for property “sample_type”.'}, {'title': 'Many-to-one relationship conflict', 'description': '"[Demo_sampleFIXED.tsv: line 113]": associated with multiple “participant” nodes: ["3BW6YS", "Y0T0JD"].'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|  97 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 101] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  98 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 112] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  99 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 100] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 100 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 111] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 101 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 110] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 102 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 99] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 103 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 98] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 104 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 109] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 105 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 97] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 106 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 107] "DNA" is not a permissible value for property “sample_type”.'}, {'title': 'Many-to-one relationship conflict', 'description': '"[Demo_sampleFIXED.tsv: line 107]": associated with multiple “participant” nodes: ["SUCWJ9", "0RR2HE"].'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| 107 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 96] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 108 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 7] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 109 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 4] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 110 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 3] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 111 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 2] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 112 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 154] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 113 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 147] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 114 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 153] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 115 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 146] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 116 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 152] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 117 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 155] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 118 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 145] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 119 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 151] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 120 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 150] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 121 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 144] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 122 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 149] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 123 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 143] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 124 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 71] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 125 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 35] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 126 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 70] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 127 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 21] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 128 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 69] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 129 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 34] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 130 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 33] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 131 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 68] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 132 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 67] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 133 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 32] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 134 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 66] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 135 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 31] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 136 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 65] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 137 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 64] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 138 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 30] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 139 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 28] "RNA" is not a permissible value for property “sample_type”.'}, {'title': 'Many-to-one relationship conflict', 'description': '"[Demo_sampleFIXED.tsv: line 28]": associated with multiple “participant” nodes: ["0RR2HE", "N12K3Y"].'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| 140 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 63] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 141 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 27] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 142 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 62] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 143 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 26] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 144 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 61] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 145 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 60] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 146 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 59] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 147 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 25] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 148 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 24] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 149 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 58] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 150 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 23] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 151 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 57] "DNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 152 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | sample       | [{'title': 'Value not permitted', 'description': '[Demo_sampleFIXED.tsv: line 22] "RNA" is not a permissible value for property “sample_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 153 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 27] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 27] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 27] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 27] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 154 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 16] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 16] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 16] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 16] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 155 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 119] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 119] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 119] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 119] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 156 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 113] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 113] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 113] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 113] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 157 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 18] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 18] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 18] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 18] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 158 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 149] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 149] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 149] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 149] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 159 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 51] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 51] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 51] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 51] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 160 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 118] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 118] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 118] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 118] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 161 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 22] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 22] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 22] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 22] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 162 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 44] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 44] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 44] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 44] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 163 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 33] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 33] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 33] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 33] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 164 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 58] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 58] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 58] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 58] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 165 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 76] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 76] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 76] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 76] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 166 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 98] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 98] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 98] "PolyA" is not a permissible value for property “library_selection”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 98] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 98] All related node IDs are missing. Please ensure at least one related node ID is included.'}]      |
| 167 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 143] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 143] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 143] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 143] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 168 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 80] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 80] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 80] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 80] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 169 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 62] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 62] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 62] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 62] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 170 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 97] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 97] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 97] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 97] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 171 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 101] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 101] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 101] "PolyA" is not a permissible value for property “library_selection”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 101] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 101] All related node IDs are missing. Please ensure at least one related node ID is included.'}] |
| 172 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 68] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 68] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 68] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 68] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 173 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 85] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 85] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 85] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 85] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 174 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 13] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 13] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 13] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 13] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 175 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 10] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 10] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 10] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 10] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 176 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 49] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 49] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 49] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 49] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 177 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 89] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 89] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 89] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 89] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 178 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 21] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 21] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 21] "PolyA" is not a permissible value for property “library_selection”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 21] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 21] All related node IDs are missing. Please ensure at least one related node ID is included.'}]      |
| 179 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 92] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 92] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 92] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 92] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 180 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 137] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 137] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 137] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 137] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 181 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 36] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 36] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 36] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 36] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 182 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 124] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 124] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 124] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 124] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 183 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 35] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 35] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 35] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 35] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 184 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 122] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 122] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 122] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 122] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 185 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 45] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 45] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 45] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 45] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 186 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 14] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 14] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 14] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 14] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 187 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 20] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 20] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 20] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 20] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 188 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 144] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 144] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 144] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 144] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 189 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 6] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 6] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 6] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 6] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                    |
| 190 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 83] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 83] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 83] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 83] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 191 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 93] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 93] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 93] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 93] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 192 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 9] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 9] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 9] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 9] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                           |
| 193 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 77] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 77] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 77] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 77] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 194 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 43] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 43] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 43] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 43] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 195 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 134] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 134] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 134] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 134] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 196 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 39] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 39] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 39] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 39] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 197 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 79] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 79] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 79] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 79] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 198 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 37] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 37] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 37] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 37] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 199 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 48] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 48] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 48] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 48] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 200 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 29] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 29] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 29] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 29] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 201 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 123] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 123] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 123] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 123] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 202 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 145] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 145] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 145] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 145] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 203 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 25] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 25] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 25] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 25] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 204 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 50] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 50] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 50] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 50] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 205 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 105] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 105] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 105] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 105] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 206 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 111] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 111] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 111] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 111] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 207 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 139] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 139] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 139] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 139] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 208 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 114] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 114] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 114] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 114] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 209 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 52] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 52] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 52] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 52] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 210 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 17] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 17] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 17] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 17] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 211 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 47] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 47] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 47] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 47] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 212 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 95] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 95] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 95] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 95] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 213 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 82] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 82] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 82] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 82] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 214 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 94] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 94] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 94] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 94] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 215 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 121] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 121] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 121] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 121] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 216 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 127] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 127] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 127] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 127] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 217 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 88] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 88] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 88] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 88] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 218 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 109] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 109] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 109] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 109] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 219 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 146] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 146] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 146] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 146] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 220 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 140] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 140] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 140] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 140] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 221 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 112] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 112] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 112] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 112] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 222 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 57] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 57] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 57] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 57] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 223 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 110] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 110] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 110] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 110] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 224 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 86] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 86] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 86] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 86] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 225 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 70] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 70] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 70] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 70] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 226 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 73] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 73] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 73] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 73] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 227 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 120] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 120] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 120] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 120] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 228 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 135] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 135] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 135] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 135] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 229 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 152] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 152] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 152] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 152] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 230 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 66] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 66] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 66] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 66] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 231 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 5] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 5] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 5] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 5] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                    |
| 232 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 74] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 74] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 74] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 74] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 233 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 96] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 96] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 96] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 96] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 234 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 129] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 129] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 129] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 129] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 235 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 41] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 41] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 41] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 41] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 236 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 138] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 138] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 138] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 138] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 237 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 147] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 147] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 147] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 147] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 238 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 3] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 3] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 3] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 3] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                           |
| 239 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 130] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 130] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 130] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 130] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 240 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 8] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 8] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 8] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 8] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                           |
| 241 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 142] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 142] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 142] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 142] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 242 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 115] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 115] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 115] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 115] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 243 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 106] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 106] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 106] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 106] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 244 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 72] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 72] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 72] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 72] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 245 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 53] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 53] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 53] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 53] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 246 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 30] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 30] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 30] "PolyA" is not a permissible value for property “library_selection”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 30] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 30] All related node IDs are missing. Please ensure at least one related node ID is included.'}]      |
| 247 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 46] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 46] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 46] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 46] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 248 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 117] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 117] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 117] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 117] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 249 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 81] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 81] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 81] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 81] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 250 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 87] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 87] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 87] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 87] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 251 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 59] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 59] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 59] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 59] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 252 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 91] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 91] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 91] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 91] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 253 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 90] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 90] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 90] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 90] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 254 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 32] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 32] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 32] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 32] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 255 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 24] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 24] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 24] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 24] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 256 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 75] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 75] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 75] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 75] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 257 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 61] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 61] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 61] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 61] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 258 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 131] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 131] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 131] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 131] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 259 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 42] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 42] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 42] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 42] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 260 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 28] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 28] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 28] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 28] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 261 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 64] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 64] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 64] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 64] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 262 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 60] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 60] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 60] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 60] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 263 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 71] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 71] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 71] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 71] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 264 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 31] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 31] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 31] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 31] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 265 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 78] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 78] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 78] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 78] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 266 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 38] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 38] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 38] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 38] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 267 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 125] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 125] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 125] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 125] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 268 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 4] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 4] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 4] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 4] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                    |
| 269 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 141] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 141] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 141] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 141] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 270 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 54] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 54] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 54] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 54] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 271 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 126] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 126] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 126] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 126] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 272 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 100] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 100] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 100] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 100] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 273 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 23] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 23] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 23] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 23] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 274 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 102] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 102] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 102] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 102] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 275 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 11] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 11] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 11] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 11] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 276 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 69] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 69] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 69] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 69] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 277 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 107] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 107] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 107] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 107] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 278 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 136] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 136] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 136] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 136] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 279 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 65] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 65] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 65] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 65] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 280 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 40] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 40] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 40] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 40] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 281 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 15] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 15] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 15] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 15] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 282 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 132] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 132] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 132] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 132] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 283 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 153] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 153] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 153] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 153] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 284 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 150] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 150] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 150] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 150] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 285 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 55] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 55] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 55] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 55] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 286 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 104] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 104] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 104] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 104] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 287 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 2] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 2] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 2] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 2] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                    |
| 288 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 67] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 67] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 67] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 67] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 289 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 63] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 63] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 63] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 63] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 290 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 133] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 133] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 133] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 133] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 291 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 7] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 7] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 7] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 7] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                           |
| 292 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 34] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 34] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 34] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 34] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 293 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 116] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 116] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 116] "PolyA" is not a permissible value for property “library_selection”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 116] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 116] All related node IDs are missing. Please ensure at least one related node ID is included.'}] |
| 294 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 103] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 103] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 103] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 103] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                            |
| 295 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 99] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 99] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 99] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 99] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 296 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 128] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 128] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 128] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 128] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 297 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 19] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 19] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 19] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 19] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 298 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 151] "Single-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 151] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 151] "PolyA" is not a permissible value for property “library_selection”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 151] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 151] All related node IDs are missing. Please ensure at least one related node ID is included.'}] |
| 299 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 56] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 56] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 56] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 56] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 300 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 84] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 84] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 84] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 84] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 301 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 12] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 12] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 12] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 12] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                       |
| 302 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 26] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 26] "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 26] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 26] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                |
| 303 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 108] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 108] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 108] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 108] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 304 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | genomic_info | [{'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 148] "Paired-end" is not a permissible value for property “library_layout”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 148] "GENOMIC" is not a permissible value for property “library_source”.'}, {'title': 'Value not permitted', 'description': '[Demo_genomic_info.tsv: line 148] "ILLUMINA" is not a permissible value for property “platform”.'}, {'title': 'Relationship not specified', 'description': '[Demo_genomic_info.tsv: line 148] All related node IDs are missing. Please ensure at least one related node ID is included.'}]                                                                                                                                                                   |
| 305 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 26] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 26] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 306 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 29] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 29] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 307 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 12] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 12] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 308 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 2] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 2] "Inflammatory carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| 309 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 17] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 17] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 310 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 31] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 31] "Invasive lobular carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 311 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 20] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 20] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 312 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 41] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 41] "Cystosarcoma phyllodes, NOS" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 313 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 39] "Neoplasm" is not a permissible value for property “disease_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 314 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 21] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 21] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 315 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 4] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 4] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 316 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 32] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 32] "Invasive lobular carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 317 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 11] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 11] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 318 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 8] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 8] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 319 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 19] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 19] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 320 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 25] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 25] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 321 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 3] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 3] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 322 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 24] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 24] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 323 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 10] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 10] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 324 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 35] "Neoplasm" is not a permissible value for property “disease_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 325 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 40] "Neoplasm" is not a permissible value for property “disease_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 326 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 18] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 18] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 327 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 13] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 13] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 328 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 23] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 23] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 329 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 15] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 15] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 330 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 22] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 22] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 331 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 37] "Neoplasm" is not a permissible value for property “disease_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 332 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 36] "Neoplasm" is not a permissible value for property “disease_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 333 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 6] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 6] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 334 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 5] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 5] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 335 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 27] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 27] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 336 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 7] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 7] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 337 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 28] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 28] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 338 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 38] "Neoplasm" is not a permissible value for property “disease_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 339 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 34] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 34] "Invasive lobular carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 340 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 16] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 16] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 341 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 30] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 30] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 342 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 9] "Neoplasm" is not a permissible value for property “disease_type”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| 343 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 14] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 14] "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 344 | 4ce43a10-4669-40ce-949f-3fbbfc9f2513 | Error      | diagnosis    | [{'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 33] "Neoplasm" is not a permissible value for property “disease_type”.'}, {'title': 'Value not permitted', 'description': '[Demo_diagnosis.tsv: line 33] "Invasive lobular carcinoma" is not a permissible value for property “primary_diagnosis”.'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |

The errors we want to summarize are lists in the **errors** column.  One of the items we do want to capture as well is the **type** column as that tells us which of the metadata submission forms contains the specific error.

In [24]:
error_list = []
for index, row in suberrors_df.iterrows():
    node = row['type']
    for entry in row['errors']:
        error_list.append({'node':node, 'title':entry['title'], 'description':entry['description']})
error_summary_df = pd.DataFrame(error_list)

In [25]:
fig = px.bar(error_summary_df['title'].value_counts())
fig.show()

Clearly *Value not permitted* is the biggest obstacle, so drilling down into that might highlight some obvious fixes.

In [26]:
vnp_df = error_summary_df.query('title == "Value not permitted"')

Because the error messages indicate the file and line, all of them are unique, so a little parsing is in order to get to somethign more informative

In [27]:
columns = ['Error', 'File', 'Line']
error_df = pd.DataFrame(columns=columns)

for index, row in vnp_df.iterrows():
    rowlist = row['description'].split(']')
    error = rowlist[-1]
    fileline = rowlist[0]
    filelist = fileline.split('[')
    filelist2 = filelist[-1].split(':')
    filename = filelist2[0]
    line = (filelist2[-1].split(' '))[-1]
    error_df.loc[len(error_df)] = [error, filename, line]


In [28]:
display(Markdown(error_df['Error'].value_counts().to_markdown()))

| Error                                                                                      |   count |
|:-------------------------------------------------------------------------------------------|--------:|
| "ILLUMINA" is not a permissible value for property “platform”.                             |     152 |
| "Paired-end" is not a permissible value for property “library_layout”.                     |     132 |
| "TRANSCRIPTOMIC" is not a permissible value for property “library_source”.                 |      77 |
| "DNA" is not a permissible value for property “sample_type”.                               |      76 |
| "RNA" is not a permissible value for property “sample_type”.                               |      76 |
| "GENOMIC" is not a permissible value for property “library_source”.                        |      75 |
| "Neoplasm" is not a permissible value for property “disease_type”.                         |      40 |
| "Invasive mammary carcinoma" is not a permissible value for property “primary_diagnosis”.  |      27 |
| "Single-end" is not a permissible value for property “library_layout”.                     |      20 |
| "PolyA" is not a permissible value for property “library_selection”.                       |       6 |
| "Invasive lobular carcinoma" is not a permissible value for property “primary_diagnosis”.  |       4 |
| "Inflammatory carcinoma" is not a permissible value for property “primary_diagnosis”.      |       1 |
| "Cystosarcoma phyllodes, NOS" is not a permissible value for property “primary_diagnosis”. |       1 |

It can also be useful to see if there are specific files that are problematic and need attention.

In [29]:
display(Markdown(error_df['File'].value_counts().to_markdown()))

| File                  |   count |
|:----------------------|--------:|
| Demo_genomic_info.tsv |     462 |
| Demo_sampleFIXED.tsv  |     152 |
| Demo_diagnosis.tsv    |      73 |